# Laboratorio 2

- Jaime Andres Torres Bermejo - 202014866
- Elkin Rafael Cuello - 202215037

## Problema 1: Redes de Transporte

La temporada navideña es uno de los periodos de mayor actividad
comercial en Colombia. Durante este tiempo, la demanda de productos como juguetes,
ropa, electrodomésticos, y alimentos tradicionales aumenta considerablemente en
diversas regiones del país. Dos de las principales ciudades distribuidoras, Bogotá y
Medellín, se preparan para abastecer a otras ciudades importantes como Cali,
Barranquilla, Pasto, Tunja, Chia y Manizales.
Cada ciudad de destino tiene una demanda específica de productos para satisfacer las
necesidades de sus habitantes durante la temporada. Mientras, Bogotá y Medellín
tienen una oferta limitada de productos por las restricciones en la cadena de suministro.
La eficiencia en la distribución es crucial para garantizar que todos los productos lleguen
a tiempo y se minimicen los costos de transporte.

### Entregables

- Plantee el modelo matematico del problema, Defina:
    - Conjuntos
    - Parámetros
    - Variable de Decisión
    - Función Objetivo
    - Restricciones
- Realize una implementación en PYOMO o en el software de su gusto
- Realize un análisis de Sensibilidad de las cuidades de destino y las cuidades de
origen.
- Mueva 50 toneladas de oferta de medellin a bogota y repita el analisis de
sensibilidad. ¿Que ha cambiado? Recomendaría este cambio or que otro cambio
puede proponer?

In [24]:
from pyomo.environ import *
import math
import numpy as np


# Modelo
model = ConcreteModel()

# Conjuntos
origenes = ['Bogota', 'Medellin']
destinos = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chia', 'Manizales']
costos = {
    ('Bogota', 'Cali'): 100, ('Medellin', 'Cali'): 2.5,
    ('Bogota', 'Barranquilla'): 2.5, ('Medellin', 'Barranquilla'): 100,
    ('Bogota', 'Pasto'): 1.6, ('Medellin', 'Pasto'): 2.0,
    ('Bogota', 'Tunja'): 1.4, ('Medellin', 'Tunja'): 1.0,
    ('Bogota', 'Chia'): 0.8, ('Medellin', 'Chia'): 1.0,
    ('Bogota', 'Manizales'): 1.4, ('Medellin', 'Manizales'): 0.8
}
demandas = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chia': 225, 'Manizales': 200}
ofertas = {'Bogota': 550, 'Medellin': 700}

# Variables
model.x = Var(origenes, destinos, within=NonNegativeReals)

# Función objetivo: minimizar los costos de transporte
def objetivo(model):
    return sum(model.x[i, j] * costos[i, j] for i in origenes for j in destinos)
model.objetivo = Objective(rule=objetivo, sense=minimize)

# Restricción de oferta: no exceder la cantidad disponible en cada origen
def oferta_rule(model, i):
    return sum(model.x[i, j] for j in destinos) <= ofertas[i]
model.oferta = Constraint(origenes, rule=oferta_rule)

# Restricción de demanda: satisfacer la demanda en cada destino
def demanda_rule(model, j):
    return sum(model.x[i, j] for i in origenes) == demandas[j]
model.demanda = Constraint(destinos, rule=demanda_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

model.display()

Model unknown

  Variables:
    x : Size=12, Index={Bogota, Medellin}*{Cali, Barranquilla, Pasto, Tunja, Chia, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogota', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Chia') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogota', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellin', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellin', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell

## Analisis de sensibilidad

In [25]:
# Cargar duales
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo
resultado = solver.solve(model)

In [26]:
# Mostrar resultados:
print("\nAnalisis de sensibilidad")

for i in destinos:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.demanda[i]]}")

for j in origenes:  # Asumo que quieres iterar solo una vez
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.oferta[j]]}")


Analisis de sensibilidad
Dual de la restricción de demanda en Cali: 2.5
Dual de la restricción de demanda en Barranquilla: 2.7
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chia: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogota: -0.2
Dual de la restricción de oferta en Medellin: 0.0


Se modifican los parametros

In [27]:
ofertas['Bogota'] = ofertas['Bogota'] + 500
ofertas['Medellin'] = ofertas['Medellin'] - 500

In [28]:
resultado = solver.solve(model)

In [29]:
# Mostrar resultados:
print("\nAnalisis de sensibilidad")

for i in destinos:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.demanda[i]]}")

for j in origenes:  # Asumo que quieres iterar solo una vez
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.oferta[j]]}")


Analisis de sensibilidad
Dual de la restricción de demanda en Cali: 2.5
Dual de la restricción de demanda en Barranquilla: 2.7
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chia: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogota: -0.2
Dual de la restricción de oferta en Medellin: 0.0
